# ws 06 IBM stock

download - https://www.kaggle.com/thebrownviking20/intro-to-recurrent-neural-networks-lstm-gru/data?select=IBM_2006-01-01_to_2018-01-01.csv

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout #

In [ ]:
df = pd.read_csv('./datasets/IBM_2006-01-01_to_2018-01-01.csv',index_col='Date')
df.index = pd.to_datetime(df.index)

df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.Close

df.head()


In [ ]:
df[:'2016'].plot(figsize=(10, 3.5),legend=True) 
df['2017':].plot(figsize=(10, 3.5),legend=True)
plt.legend(['Training set (before 2017)','Test set (since 2017)'])
plt.title('IBM stock price')

plt.show()

In [ ]:
print('Samples:', df.shape)
print('Train set:', df[:'2016'].shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

data = df.values.reshape(-1, 1)
sc = MinMaxScaler()
data_sc = sc.fit_transform(data)
data_sc[:5]

In [ ]:
def convertToMatrix(data, step=1):
    X, Y = [], []
    for i in range(len(data)-step):
        d = i + step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

In [ ]:
n_train = 2769

train, test = data_sc[0:n_train], data_sc[n_train:]
train.shape, test.shape

In [ ]:
step = 60 

X_train, y_train = convertToMatrix(train, step)  
X_test, y_test = convertToMatrix(test, step)

print('X Train, Test (after conversion):', X_train.shape, X_test.shape)
print('y Train, Test (after conversion):', y_train.shape, y_test.shape)

### Model

In [ ]:
model = Sequential()

# model.add(LSTM(units=50, return_sequences=True, input_shape=(step, 1), activation='relu'))
# model.add(Dropout(0.2))
# model.add(LSTM(units=40, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(units=1))  # output

# GRU
model.add(GRU(units=50, return_sequences=True, input_shape=(step, 1), activation='relu'))
model.add(Dropout(0.2))
model.add(GRU(units=40, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.summary()

In [ ]:
history = model.fit(X_train, y_train,epochs=50, batch_size=32, verbose=1)

In [ ]:
plt.plot(history.history['loss'], label='Train loss')
plt.legend()
plt.show()

In [ ]:
testPredict = model.predict(X_test) 

testPredict.shape

In [ ]:
y_test.shape

In [ ]:
print(y_train.shape)
sc.inverse_transform(y_train.reshape(-1,1))[:5]

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

testPredict_inv = sc.inverse_transform(testPredict)

y_test_inv = sc.inverse_transform(y_test)

r2_test = r2_score(y_test_inv, testPredict_inv) #

rmse_test = np.sqrt(mean_squared_error(y_test_inv, testPredict_inv))

print('R2 Test: {:.3f}'. format(r2_test))
print('RMSE Test: {:.3f}'. format(rmse_test))

In [ ]:
plt.figure(figsize=(12-4,3.1))

plt.plot(y_test_inv, lw=1.3, label='Dataset (Test)')

plt.plot(testPredict_inv, 'r', lw=2.4, label='Prediction')
plt.legend()
plt.show()